# 图像回归

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

In [2]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu1 = gpus[1] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu1, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu1],"GPU") 

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

为了使本教程易于理解，我们仅将 MNIST 数据集视为回归数据集。 这意味着我们将把 MNIST 数据集的预测目标，即 0 到 9 之间的整数视为数值，以便它们可以直接用作回归目标。

## 一个简单的例子

第一步是准备数据。 这里我们以MNIST数据集为例

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:100]
y_train = y_train[:100]
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

(100, 28, 28)
(100,)
[5 0 4]


第二步是运行 ImageRegressor。 建议对更复杂的数据集进行更多试验。 这只是 MNIST 的一个快速演示，因此我们将 max_trials 设置为 1。出于同样的原因，我们将 epochs 设置为 2。您也可以为自适应数量的 epochs 不指定 epochs。

In [5]:
# Initialize the image regressor.
reg = ak.ImageRegressor(overwrite=True, max_trials=1)
# Feed the image regressor with training data.
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 32s]
val_loss: 9.849075317382812

Best val_loss So Far: 9.849075317382812
Total elapsed time: 00h 00m 32s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 9s 1s/step - loss: 29.5317 - mean_squared_error: 29.5317
Epoch 2/2
4/4 [==============================] - 0s 50ms/step - loss: 47.9474 - mean_squared_error: 47.9474


/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./image_regressor/best_model/assets


In [6]:
# Predict with the best model.
predicted_y = reg.predict(x_test)
print(predicted_y)

313/313 [==============================] - 5s 15ms/step
[[2.491232 ]
 [2.9314375]
 [2.3841684]
 ...
 [3.5095334]
 [2.814288 ]
 [3.2001033]]


In [7]:
# Evaluate the best model with testing data.
print(reg.evaluate(x_test, y_test))

313/313 [==============================] - 6s 17ms/step - loss: 10.0003 - mean_squared_error: 10.0003
[10.00033950805664, 10.00033950805664]


## 验证数据

默认情况下，AutoKeras 使用最后 20% 的训练数据作为验证数据。 如下例所示，您可以使用validation_split 来指定百分比。

In [8]:
reg.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=2,
)

您还可以使用自己的验证集，而不是使用validation_data 将其从训练数据中分离出来。

In [9]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
reg.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=2,
)

## 自定义搜索空间

对于高级用户，您可以使用 AutoModel 而不是 ImageRegressor 来自定义您的搜索空间。 您可以为一些高级配置配置 ImageBlock，例如，block_type 为要搜索的神经网络类型，normalize 为是否进行数据归一化，augment 为是否进行数据增强。 您也可以不指定这些参数，这将使不同的选择自动调整。 有关详细信息，请参阅以下示例。

In [10]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=False,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 10s]
val_loss: 376242591236096.0

Best val_loss So Far: 376242591236096.0
Total elapsed time: 00h 00m 10s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 4s 39ms/step - loss: 307.1821 - mean_squared_error: 307.1821
Epoch 2/2
4/4 [==============================] - 0s 42ms/step - loss: 95201.8516 - mean_squared_error: 95201.8516


/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


AutoModel 的用法类似于 Keras 的函数式 API。 基本上，您正在构建一个图，其边是块，节点是块的中间输出。 使用 output_node = ak.some_block(input_node) 添加从 input_node 到 output_node 的边。

您甚至还可以使用更细粒度的块来进一步自定义搜索空间。 请参阅以下示例。

In [11]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
reg.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 22s]
val_loss: 28.69540786743164

Best val_loss So Far: 28.69540786743164
Total elapsed time: 00h 00m 22s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
4/4 [==============================] - 8s 1s/step - loss: 17.7086 - mean_squared_error: 17.7086
Epoch 2/2
4/4 [==============================] - 0s 43ms/step - loss: 11.3087 - mean_squared_error: 11.3087


/public/huangwei/miniconda3/envs/tensorflow/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


## 数据格式

AutoKeras ImageRegressor 对于数据格式非常灵活。

对于图像，它接受带和不带通道维度的数据格式。 MNIST 数据集中的图像没有通道维度。 每个图像都是一个形状为 (28, 28) 的矩阵。 AutoKeras 也接受最终带有通道维度的三个维度的图像，例如 (32, 32, 3), (28, 28, 1)。

对于回归目标，它应该是数值向量。 AutoKeras 接受 numpy.ndarray。

我们还支持对训练数据使用 tf.data.Dataset 格式。 在这种情况下，图像必须是 3 维的。

In [12]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.reshape(x_test.shape + (1,))
y_train = y_train.reshape(y_train.shape + (1,))
y_test = y_test.reshape(y_test.shape + (1,))

print(x_train.shape)  # (60000, 28, 28, 1)
print(y_train.shape)  # (60000, 10)

(60000, 28, 28, 1)
(60000, 1)


In [13]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,), (y_train,)))
test_set = tf.data.Dataset.from_tensor_slices(((x_test,), (y_test,)))

reg = ak.ImageRegressor(overwrite=True, max_trials=1)
# Feed the tensorflow Dataset to the regressor.
reg.fit(train_set, epochs=2)

Trial 1 Complete [00h 00m 53s]
val_loss: 0.7645319700241089

Best val_loss So Far: 0.7645319700241089
Total elapsed time: 00h 00m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/2
1875/1875 [==============================] - 19s 10ms/step - loss: 1.4795 - mean_squared_error: 1.4795
Epoch 2/2
1875/1875 [==============================] - 19s 10ms/step - loss: 0.7656 - mean_squared_error: 0.7656
INFO:tensorflow:Assets written to: ./image_regressor/best_model/assets


In [16]:
# Predict with the best model.
predicted_y = reg.predict(test_set)
predicted_y

313/313 [==============================] - 1s 3ms/step


array([[6.5926566],
       [1.8833197],
       [1.0955929],
       ...,
       [3.538729 ],
       [4.59532  ],
       [5.8301196]], dtype=float32)

In [15]:
# Evaluate the best model with testing data.
print(reg.evaluate(test_set))

313/313 [==============================] - 2s 7ms/step - loss: 1.1663 - mean_squared_error: 1.1663
[1.1662607192993164, 1.1662607192993164]
